# Various plots for output data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib qt 

import astropy.units as u

In [2]:
catalog_Bstar = pd.read_csv('crossmatch_mag_exo.csv', delimiter=',')
crossmatch = catalog_Bstar[catalog_Bstar['Simbad_ID']=='G 139-21']
if crossmatch.size > 0 :
    mag_field = np.array(crossmatch['Bestim_G'])[0] * 1e-4
else :
    mag_field = np.nan
print(mag_field)

0.0327536231884058


### NenuFAR sensitivity

In [3]:
from nenupy.instru.interferometer import ObservingMode
from nenupy.instru import NenuFAR
import astropy.units as u

NenuFAR_sensitivity = []
frequencies = np.linspace(10,80,100)
nenufar = NenuFAR()


NenuFAR_sensitivity = nenufar.sensitivity( 
    frequency=frequencies*u.MHz,
    mode=ObservingMode.BEAMFORMING,
    dt=10*60*u.s,
    df=2*u.MHz,
    elevation=90*u.deg,
    efficiency=1.,
    decoherence=1.,
    lna_filter=0)


In [19]:
nenufar._antenna_names

array(['MA000', 'MA001', 'MA002', 'MA003', 'MA004', 'MA005', 'MA006',
       'MA007', 'MA008', 'MA009', 'MA010', 'MA011', 'MA012', 'MA013',
       'MA014', 'MA015', 'MA016', 'MA017', 'MA018', 'MA019', 'MA020',
       'MA021', 'MA022', 'MA023', 'MA024', 'MA025', 'MA026', 'MA027',
       'MA028', 'MA029', 'MA030', 'MA031', 'MA032', 'MA033', 'MA034',
       'MA035', 'MA036', 'MA037', 'MA038', 'MA039', 'MA040', 'MA041',
       'MA042', 'MA043', 'MA044', 'MA045', 'MA046', 'MA047', 'MA048',
       'MA049', 'MA050', 'MA051', 'MA052', 'MA053', 'MA054', 'MA055',
       'MA056', 'MA057', 'MA058', 'MA059', 'MA060', 'MA061', 'MA062',
       'MA063', 'MA064', 'MA065', 'MA066', 'MA067', 'MA068', 'MA069',
       'MA070', 'MA071', 'MA072', 'MA073', 'MA074', 'MA075', 'MA076',
       'MA077', 'MA078', 'MA079', 'MA080', 'MA081', 'MA082', 'MA083',
       'MA084', 'MA085', 'MA086', 'MA087', 'MA088', 'MA089', 'MA090',
       'MA091', 'MA092', 'MA093', 'MA094', 'MA095'], dtype='<U5')

In [21]:
plt.figure()
plt.plot(frequencies,NenuFAR_sensitivity)
plt.xlabel('Frequencies [MHz]')
plt.ylabel('Sensitivity [Jy]')
plt.title('NenuFAR sensitivity with frequencies')
plt.grid()
plt.show()

## Loading output data

### MSB

In [4]:
data_msb = pd.read_csv(r"/Users/emauduit/Documents/These/target_selection/Runs/2024-09-03T18h11/main_output.csv", delimiter=';')
#'/Users/emauduit/Documents/These/target_selection/Runs/2024-07-04T23h06/main_output.csv',delimiter=';') V1 manuscrit

In [9]:
data_msb.keys()

Index(['name', 'planet_mass', 'planet_radius', 'planet_luminosity',
       'star_planet_distance', 'planet_rotation_rate', 'planet_orbital_period',
       'star_simbad_id', 'star_mass', 'star_radius', 'star_age',
       'earth_distance', 'star_magfield', 'star_rotperiod', 'star_luminosity',
       'spectral_type', 'spectral_type_code', 'star_effective_temp',
       'dynamo_density', 'dynamo_radius', 'B_dyn', 'B_eq', 'magnetic_moment',
       'standoff_distance', 'sw_density', 'sw_velocity', 'coronal_temperature',
       'sw_magfield', 'alfven_velocity', 'magnetic_field_planet',
       'freq_max_planet', 'pow_emission_kinetic', 'pow_emission_magnetic',
       'pow_emission_spi', 'flux_kinetic_au', 'flux_magnetic_au',
       'flux_spi_au', 'pow_received_kinetic', 'pow_received_magnetic',
       'pow_received_spi', 'freq_max_star'],
      dtype='object')

In [5]:
target_name = np.array(data_msb['name'][1:],dtype='str')
Pkin_msb = np.array(data_msb['pow_received_kinetic'][1:],dtype='float')
Pmag_msb = np.array(data_msb['pow_received_magnetic'][1:],dtype='float')
Pspi_msb = np.array(data_msb['pow_received_spi'][1:],dtype='float')

fmax_planet_msb = np.array(data_msb['freq_max_planet'][1:],dtype='float')
fmax_star_msb = np.array(data_msb['freq_max_star'][1:],dtype='float')

star_age_msb = np.array(data_msb['star_age'][1:], dtype='float')
Mp_msb = np.array(data_msb['planet_mass'][1:], dtype='float')
Rp_msb = np.array(data_msb['planet_radius'][1:], dtype='float')
s_msb = np.array(data_msb['earth_distance'][1:], dtype='float')
d_msb = np.array(data_msb['star_planet_distance'][1:], dtype='float')
sp_type_msb = np.array(data_msb['spectral_type'][1:], dtype='str')


mask_age = star_age_msb != 5.2e9

In [6]:
output_msb = pd.DataFrame({'name' : target_name,
                'planet_mass' : Mp_msb,
                'planet_radius': Rp_msb,
                'star_planet_distance' : d_msb,
                'star_age' : star_age_msb,
                'spectral_type' : sp_type_msb,
                'earth_distance' : s_msb,
                'pow_received_kinetic' : Pkin_msb, 
                'pow_received_magnetic' : Pmag_msb, 
                'freq_max_planet':fmax_planet_msb,
                'pow_received_spi' : Pspi_msb,  
                'freq_max_star' : fmax_star_msb})

In [ ]:
output_msb[:10]

- Pkin

In [13]:
plt.figure(figsize=(10,7))
plt.scatter(fmax_planet_msb[~np.isnan(Pkin_msb)],Pkin_msb[~np.isnan(Pkin_msb)], marker='+', color ='tab:blue', label='P_kin')
plt.plot([10,10],[np.min(Pkin_msb[~np.isnan(Pkin_msb)]),np.max(Pkin_msb[~np.isnan(Pkin_msb)])],linestyle='dashed', color='black')
plt.xlabel('Maximum frequency emission [MHz]')
plt.ylabel('Expected kinetic flux at Earth [mJy]')
plt.title('Predicted radio flux with respect to maximum frequency.')
plt.xscale('log')
plt.yscale('log')
plt.grid()
plt.legend()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/emission/Pkin_vs_fmax.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

- Pmag

In [9]:
fig,ax = plt.subplots(figsize=(10,7))
ax.scatter(fmax_planet_msb[~np.isnan(Pmag_msb)], Pmag_msb[~np.isnan(Pmag_msb)], marker='+', color='tab:blue')
#telescopes
ax.plot(frequencies,NenuFAR_sensitivity*1e3, color='tab:red', label='NenuFAR')
#ax.text(40,8,'NenuFAR', color='tab:red', fontsize=14)
ax.plot([14,19,30,60,75],[2.5e2,1e2,4e1,2e1,5e1], color = 'goldenrod', label='LOFAR')
ax.plot([130,160,190,200,250],[1,9e-1,1,2,6], color = 'goldenrod')
#ax.text(115,80, 'LOFAR', color='goldenrod', fontsize=14)
ax.plot([10,30],[1e1,1e1], color='forestgreen', label='UTR-2')
#ax.text(15,5, 'UTR-2', color='forestgreen', fontsize=14)
ax.plot([50,90,200,350],[5e-1,3e-1,1.5e-1,6e-2], color='purple', label='SKA 1 low')
#ax.text(350,8e-2,'SKA 1 low', color='purple', fontsize=14)
ax.plot([50,90,200,350],[5e-2,3e-2,1.5e-2,6e-3], color='purple', linestyle='dashed', label = 'SKA 2 low')
#ax.text(350,8e-3,'SKA 2 low', color='purple', fontsize=14)
ax.plot([70,80],[9e1,9e1], linewidth=5, color='darkcyan', label='VLA')
#ax.text(70,1e2, 'VLA', color='darkcyan',fontsize=14)
ax.scatter(150,3, color='y', label='GMRT')
#ax.text(140,4, 'GMRT', color='y', fontsize=14)

ax.plot([10.0,10.0],[1e-9,1e10], linestyle = 'dashed', color='black',label='coupure\nionosphérique')
rect = plt.Rectangle((1e-2,1e-9),10-1e-2,1e19,facecolor='black',alpha=0.1)
ax.add_patch(rect)
ax.set_xlabel('$f_{c,p}^{max}$  [MHz]', fontsize=18)
ax.set_ylabel('$\Phi_{radio}^{mag}$  [mJy]', fontsize=18)
ax.tick_params(axis='both',labelsize=14)
ax.set_xlim(1e-2,3e4)
ax.set_ylim(1e-9,1e6)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('MSB')
plt.grid()
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/ch4/fig/plot_Pmag_fp_MSB.png',transparent=True,bbox_inches='tight', dpi=150 )
plt.show()



In [19]:
plt.figure(figsize=(10,7))
plt.scatter(fmax_planet_msb[~np.isnan(Pmag_msb)], Pmag_msb[~np.isnan(Pmag_msb)], marker='+', color='tab:red',label='P_mag')
plt.plot([10,10],[np.min(Pmag_msb[~np.isnan(Pmag_msb)]),np.max(Pmag_msb[~np.isnan(Pmag_msb)])],linestyle='dashed', color='black')
plt.xlabel('Maximum frequency emission [MHz]')
plt.ylabel('Expected magnetic flux at Earth [mJy]')
plt.title('Predicted radio flux with respect to maximum frequency.')
plt.xscale('log')
plt.yscale('log')
plt.grid()
plt.legend()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/emission/Pmag_vs_fmax_msb.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

- Pspi

In [8]:
fig,ax = plt.subplots(figsize=(10,7))
ax.scatter(fmax_star_msb[~np.isnan(Pspi_msb)], Pspi_msb[~np.isnan(Pspi_msb)], marker='+', color='tab:blue')#,label='$\Phi_{radio}^{SPI}$')
#telescopes
ax.plot(frequencies,NenuFAR_sensitivity*1e3, color='tab:red', label='NenuFAR')
#ax.text(40,8,'NenuFAR', color='tab:red', fontsize=14)
ax.plot([14,19,30,60,75],[2.5e2,1e2,4e1,2e1,5e1], color = 'goldenrod', label='LOFAR')
ax.plot([130,160,190,200,250],[1,9e-1,1,2,6], color = 'goldenrod')
#ax.text(115,80, 'LOFAR', color='goldenrod', fontsize=14)
ax.plot([10,30],[1e1,1e1], color='forestgreen', label='UTR-2')
#ax.text(15,5, 'UTR-2', color='forestgreen', fontsize=14)
ax.plot([50,90,200,350],[5e-1,3e-1,1.5e-1,6e-2], color='purple', label='SKA 1 low')
#ax.text(350,8e-2,'SKA 1 low', color='purple', fontsize=14)
ax.plot([50,90,200,350],[5e-2,3e-2,1.5e-2,6e-3], color='purple', linestyle='dashed', label = 'SKA 2 low')
#ax.text(350,8e-3,'SKA 2 low', color='purple', fontsize=14)
ax.plot([70,80],[9e1,9e1], linewidth=5, color='darkcyan', label='VLA')
#ax.text(70,1e2, 'VLA', color='darkcyan',fontsize=14)
ax.scatter(150,3, color='y', label='GMRT')
#ax.text(140,4, 'GMRT', color='y', fontsize=14)

ax.plot([10.0,10.0],[1e-9,1e10], linestyle = 'dashed', color='black')
rect = plt.Rectangle((1e-2,1e-9),10-1e-2,1e19,facecolor='black',alpha=0.1,label='coupure\nionosphérique')
ax.add_patch(rect)
ax.set_xlabel('$f_{c,*}^{max}$  [MHz]', fontsize=18)
ax.set_ylabel('$\Phi_{radio}^{SPI}$  [mJy]', fontsize=18)
ax.tick_params(axis='both',labelsize=14)
ax.set_xlim(1e-2,3e4)
ax.set_ylim(1e-9,1e6)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('MSB')
plt.grid()
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/ch4/fig/plot_Pspi_fstar_MSB.png', transparent=True, bbox_inches='tight', dpi=150 )
plt.show()

In [45]:
plt.figure(figsize=(10,7))
plt.scatter(fmax_star_msb[~np.isnan(Pspi_msb) & mask_age],Pspi_msb[~np.isnan(Pspi_msb) & mask_age ],marker='+', color='tab:green', label='P_spi')
plt.plot([10,10],[np.min(Pspi_msb[~np.isnan(Pspi_msb) & mask_age ]),np.max(Pspi_msb[~np.isnan(Pspi_msb) & mask_age])],linestyle='dashed', color='black')
plt.xlabel('Maximum frequency emission [MHz]')
plt.ylabel('Expected SPI flux at Earth [mJy]')
plt.title('Predicted radio flux with respect to maximum frequency.')
plt.xscale('log')
plt.yscale('log')
plt.grid()
plt.legend()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/emission/Pspi_vs_fmax.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

- all

In [46]:
plt.figure(figsize=(10,7))
#plt.scatter(fmax_planet[~np.isnan(Pkin)],Pkin[~np.isnan(Pkin)], marker='+', color ='tab:blue', label='P_kin')
plt.scatter(fmax_planet_msb[~np.isnan(Pmag_msb)], Pmag_msb[~np.isnan(Pmag_msb)], marker='x', color='tab:blue',label='P_mag')
plt.scatter(fmax_star_msb[~np.isnan(Pspi_msb) & mask_age],Pspi_msb[~np.isnan(Pspi_msb) & mask_age],marker='x', color='tab:orange', label='P_spi')
plt.plot(frequencies,NenuFAR_sensitivity*1e3, linestyle='dashed', color='black', label='NenuFAR sensitivity')
plt.plot([10,10],[np.min(Pmag_msb[~np.isnan(Pmag_msb)]),np.max(Pmag_msb[~np.isnan(Pmag_msb)])], color='black', label='Ionospheric cut off')
plt.xlabel('Maximum frequency emission [MHz]')
plt.ylabel('Expected flux at Earth [mJy]')
plt.title('Predicted radio flux with respect to maximum frequency.')
plt.xscale('log')
plt.yscale('log')
plt.grid()
plt.legend()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/emission/P_vs_fmax.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

Pic à 3.6 MHz en SPI :
Vient du forçage de t*=5.2 Gyr si on ne connait pas l'âge de l'étoile

In [ ]:
tau = 2.56e7  # yr
K = 0.6709505359255223
rotperiod = K * pow(1 + (5.2e9 / tau), 0.7)
print(rotperiod)
Psun = 25.5  # days
Bsun = 1.435e-4  # T
magfield = Bsun * Psun / rotperiod

me = 9.1093897e-31  # kg
e = 1.60217733e-19  # C
freq = e * magfield / (2 * np.pi * me)
print(freq/1e6)

### RC

In [10]:
data_rc = pd.read_csv(r'/Users/emauduit/Documents/These/target_selection/Runs/2024-09-03T18h18/main_output.csv',delimiter=';')
#'/Users/emauduit/Documents/These/target_selection/Runs/2024-07-04T23h08/main_output.csv',delimiter=';')

In [11]:
target_name = np.array(data_rc['name'][1:],dtype='str')
Pkin_rc = np.array(data_rc['pow_received_kinetic'][1:],dtype='float')
Pmag_rc = np.array(data_rc['pow_received_magnetic'][1:],dtype='float')
Pspi_rc = np.array(data_rc['pow_received_spi'][1:],dtype='float')

fmax_planet_rc = np.array(data_rc['freq_max_planet'][1:],dtype='float')
fmax_star_rc = np.array(data_rc['freq_max_star'][1:],dtype='float')

star_age_rc = np.array(data_rc['star_age'][1:], dtype='float')
Mp_rc = np.array(data_rc['planet_mass'][1:], dtype='float')
Rp_rc = np.array(data_rc['planet_radius'][1:], dtype='float')
s_rc = np.array(data_rc['earth_distance'][1:], dtype='float')
d_rc = np.array(data_rc['star_planet_distance'][1:], dtype='float')
sp_type_rc = np.array(data_rc['spectral_type'][1:], dtype='str')

mask_age_rc = star_age_rc != 5.2e9

In [12]:
output_rc = pd.DataFrame({'name' : target_name,
                'planet_mass' : Mp_rc,
                'planet_radius': Rp_rc,
                'star_planet_distance' : d_rc,
                'star_age' : star_age_rc,
                'spectral_type' : sp_type_rc,
                'earth_distance' : s_rc,
                'pow_received_kinetic' : Pkin_rc, 
                'pow_received_magnetic' : Pmag_rc, 
                'freq_max_planet':fmax_planet_rc,
                'pow_received_spi' : Pspi_rc,  
                'freq_max_star' : fmax_star_rc})

In [13]:
fig,ax = plt.subplots(figsize=(10,7))
ax.scatter(fmax_planet_rc[~np.isnan(Pmag_rc)], Pmag_rc[~np.isnan(Pmag_rc)], marker='+', color='tab:blue')#,label='$\Phi_{radio}^{mag}$')
#telescopes
ax.plot(frequencies,NenuFAR_sensitivity*1e3, color='tab:red', label='NenuFAR')
#ax.text(40,8,'NenuFAR', color='tab:red', fontsize=14)
ax.plot([14,19,30,60,75],[2.5e2,1e2,4e1,2e1,5e1], color = 'goldenrod', label='LOFAR')
ax.plot([130,160,190,200,250],[1,9e-1,1,2,6], color = 'goldenrod')
#ax.text(115,80, 'LOFAR', color='goldenrod', fontsize=14)
ax.plot([10,30],[1e1,1e1], color='forestgreen', label='UTR-2')
#ax.text(15,5, 'UTR-2', color='forestgreen', fontsize=14)
ax.plot([50,90,200,350],[5e-1,3e-1,1.5e-1,6e-2], color='purple', label='SKA 1 low')
#ax.text(350,8e-2,'SKA 1 low', color='purple', fontsize=14)
ax.plot([50,90,200,350],[5e-2,3e-2,1.5e-2,6e-3], color='purple', linestyle='dashed', label = 'SKA 2 low')
#ax.text(350,8e-3,'SKA 2 low', color='purple', fontsize=14)
ax.plot([70,80],[9e1,9e1], linewidth=5, color='darkcyan', label='VLA')
#ax.text(70,1e2, 'VLA', color='darkcyan',fontsize=14)
ax.scatter(150,3, color='y', label='GMRT')
#ax.text(140,4, 'GMRT', color='y', fontsize=14)

ax.plot([10.0,10.0],[1e-9,1e10], linestyle = 'dashed', color='black',label='coupure\nionosphérique')
rect = plt.Rectangle((1e-2,1e-9),10-1e-2,1e19,facecolor='black',alpha=0.1)
ax.add_patch(rect)
ax.set_xlabel('$f_{c,p}^{max}$  [MHz]', fontsize=18)
ax.set_ylabel('$\Phi_{radio}^{mag}$  [mJy]', fontsize=18)
ax.tick_params(axis='both',labelsize=14)
ax.set_xlim(1e-2,3e4)
ax.set_ylim(1e-9,1e6)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('RC')
plt.grid()
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/ch4/fig/plot_Pmag_fp_RC.png',transparent=True,bbox_inches='tight', dpi=150 )
plt.show()

In [14]:
fig,ax = plt.subplots(figsize=(10,7))
ax.scatter(fmax_star_rc[~np.isnan(Pspi_rc)], Pspi_rc[~np.isnan(Pspi_rc)], marker='+', color='tab:blue')#,label='$\Phi_{radio}^{SPI}$')
#telescopes
ax.plot(frequencies,NenuFAR_sensitivity*1e3, color='tab:red', label='NenuFAR')
#ax.text(40,8,'NenuFAR', color='tab:red', fontsize=14)
ax.plot([14,19,30,60,75],[2.5e2,1e2,4e1,2e1,5e1], color = 'goldenrod', label='LOFAR')
ax.plot([130,160,190,200,250],[1,9e-1,1,2,6], color = 'goldenrod')
#ax.text(115,80, 'LOFAR', color='goldenrod', fontsize=14)
ax.plot([10,30],[1e1,1e1], color='forestgreen', label='UTR-2')
#ax.text(15,5, 'UTR-2', color='forestgreen', fontsize=14)
ax.plot([50,90,200,350],[5e-1,3e-1,1.5e-1,6e-2], color='purple', label='SKA 1 low')
#ax.text(350,8e-2,'SKA 1 low', color='purple', fontsize=14)
ax.plot([50,90,200,350],[5e-2,3e-2,1.5e-2,6e-3], color='purple', linestyle='dashed', label = 'SKA 2 low')
#ax.text(350,8e-3,'SKA 2 low', color='purple', fontsize=14)
ax.plot([70,80],[9e1,9e1], linewidth=5, color='darkcyan', label='VLA')
#ax.text(70,1e2, 'VLA', color='darkcyan',fontsize=14)
ax.scatter(150,3, color='y', label='GMRT')
#ax.text(140,4, 'GMRT', color='y', fontsize=14)

ax.plot([10.0,10.0],[1e-9,1e11], linestyle = 'dashed', color='black', label='coupure\nionosphérique')
rect = plt.Rectangle((1e-2,1e-9),10-1e-2,1e19,facecolor='black',alpha=0.1)
ax.add_patch(rect)
ax.set_xlabel('$f_{c,*}^{max}$ [MHz]', fontsize=18)
ax.set_ylabel('$\Phi_{radio}^{SPI}$ [mJy]', fontsize=18)
ax.tick_params(axis='both',labelsize=14)
ax.set_xlim(1e-2,3e4)
ax.set_ylim(1e-9,1e6)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('RC')
plt.grid()
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/ch4/fig/plot_Pspi_fstar_RC.png',transparent=True,bbox_inches='tight', dpi=150 )
plt.show()

## Sélection de cibles

In [66]:
data_msb_mag_sorted=output_msb.sort_values(by = ["pow_received_magnetic"], ascending=[False])
data_msb_mag_small=data_msb_mag_sorted[data_msb_mag_sorted['freq_max_planet'] >= 4.99].iloc[0:100]
data_msb_spi_sorted=output_msb.sort_values(by = ["pow_received_spi"], ascending=[False])
data_msb_spi_small=data_msb_spi_sorted[data_msb_spi_sorted['freq_max_star'] >= 4.99].iloc[0:100]

In [67]:
data_msb_mag_small.keys()

Index(['name', 'planet_mass', 'planet_radius', 'star_planet_distance',
       'star_age', 'spectral_type', 'earth_distance', 'pow_received_kinetic',
       'pow_received_magnetic', 'freq_max_planet', 'pow_received_spi',
       'freq_max_star'],
      dtype='object')

In [68]:
data_msb_mag_small.to_csv('data_msb_mag_top100_new.csv')
data_msb_spi_small.to_csv('data_msb_spi_top100_new.csv')

In [69]:
data_rc_mag_sorted=output_rc.sort_values(by = ["pow_received_magnetic"], ascending=[False])
data_rc_mag_small=data_rc_mag_sorted[data_rc_mag_sorted['freq_max_planet'] >= 4.99].iloc[0:100]
data_rc_spi_sorted=output_rc.sort_values(by = ["pow_received_spi"], ascending=[False])
data_rc_spi_small=data_rc_spi_sorted[data_rc_spi_sorted['freq_max_star'] >= 4.99].iloc[0:100]

In [70]:
data_rc_mag_small.to_csv('data_rc_mag_top100_new.csv')
data_rc_spi_small.to_csv('data_rc_spi_top100_new.csv')

In [71]:
names_msb_mag = np.array(data_msb_mag_small.index, dtype='str')
names_msb_spi = np.array(data_msb_spi_small.index, dtype='str')

In [72]:
df_msb = pd.concat([data_msb_mag_small,data_msb_spi_small])
df_msb_uniq = df_msb.drop_duplicates(subset='name')
df_msb_uniq.to_csv('data_msb_top100_new.csv')

In [73]:
df_rc = pd.concat([data_rc_mag_small,data_rc_spi_small])
df_rc_uniq = df_rc.drop_duplicates(subset='name')
df_rc_uniq.to_csv('data_rc_top100_new.csv')

In [75]:
df_msb_uniq = pd.read_csv('data_msb_top100_new.csv', delimiter=',',index_col=1)
df_rc_uniq = pd.read_csv('data_rc_top100_new.csv',delimiter=',',index_col=1)

In [76]:
df_msb_uniq =  df_msb_uniq.rename(columns={'pow_received_kinetic' : 'P_kin_msb',
                    'pow_received_magnetic' : 'P_mag_msb', 'freq_max_planet' : 'f_p_msb', 'pow_received_spi' : 'P_spi_msb',
                    'freq_max_star' : 'fstar_msb'})

df_rc_uniq =  df_rc_uniq.rename(columns={'pow_received_kinetic' : 'P_kin_rc',
                    'pow_received_magnetic' : 'P_mag_rc', 'freq_max_planet' : 'f_p_rc', 'pow_received_spi' : 'P_spi_rc',
                    'freq_max_star' : 'fstar_rc'})

In [77]:
df_rc_uniq.keys()

Index(['Unnamed: 0', 'planet_mass', 'planet_radius', 'star_planet_distance',
       'star_age', 'spectral_type', 'earth_distance', 'P_kin_rc', 'P_mag_rc',
       'f_p_rc', 'P_spi_rc', 'fstar_rc'],
      dtype='object')

In [78]:
names_msb=np.array(df_msb_uniq.index)
names_rc=np.array(df_rc_uniq.index)

In [79]:
data_cross_MSB=df_msb_uniq.loc[:, ['planet_mass', 'planet_radius', 'star_planet_distance','star_age', 'spectral_type', 'earth_distance']]
data_cross_RC=df_rc_uniq.loc[:, ['planet_mass', 'planet_radius', 'star_planet_distance','star_age', 'spectral_type', 'earth_distance']]

In [80]:
data_cross_RC=data_cross_RC[~data_cross_RC.index.isin(names_msb)]
data_cross=pd.concat([data_cross_MSB,data_cross_RC])

In [81]:
data_cross['planet_mass'][0:10]

name
GJ 1214 b     0.025710
GJ 724 b      0.033820
YZ Cet b      0.002200
YZ Cet c      0.003590
GJ 486 b      0.008870
TOI-4481 b    0.006000
GJ 367  b     0.001718
GJ 1061 b     0.004530
TOI-1685 b    0.009880
TOI-732 b     0.007740
Name: planet_mass, dtype: float64

In [82]:
data_param_MSB=df_msb_uniq.loc[:, ['P_kin_msb', 'P_mag_msb','f_p_msb', 'P_spi_msb', 'fstar_msb']]
data_param_RC=df_rc_uniq.loc[:,['P_kin_rc', 'P_mag_rc','f_p_rc', 'P_spi_rc', 'fstar_rc']]

In [83]:
data_param=data_param_MSB.join(data_param_RC,how='outer')

In [84]:
data_cross=data_cross.join(data_param,how='outer')

In [85]:
data_cross.to_csv('data_all_top100_new.csv')